In [19]:
print('Inicializando Variáveis do Programa')

ano_início = 2013
ano_fim = 2020
confiança = 0.01

Inicializando Variáveis do Programa


In [20]:
from datetime import datetime
print('Início em: ', datetime.now())

print('Importações Iniciais')

from tkinter import NONE
from Lattes import Lattes
from Carga import Carga
from Database import Database
import psycopg2, pickle
import pandas, os, math
import statsmodels.api as sm
from urllib.parse import urlencode
from psycopg2.extensions import AsIs
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import Database
import json
import locale
import pylab
from scipy import stats
from scipy.stats import norm, skewnorm, shapiro, kstest
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

print('Fim: ', datetime.now())

Início em:  2023-04-27 07:02:47.695626
Importações Iniciais
Fim:  2023-04-27 07:02:47.696625


In [21]:
#Carregar arquivo na memória
print('Carregando indicadores na memória. Início em: ', datetime.now())

db = Database.Database('CNPq')
engine = Carga.db_engine()
sql = '''
select * from financiamentos_doutorandos_2014
    inner join indicadores_doutorado
        on indicadores_doutorado.id = financiamentos_doutorandos_2014.id
    inner join indicadores_nomes
        on indicadores_nomes.tipo = indicadores_doutorado.indicador_tipo                 
'''
dt = pd.read_sql(sql, engine)
print(f'dt.size: {dt.size}')

print('Término em: ', datetime.now())

Carregando indicadores na memória. Início em:  2023-04-27 07:02:48.102562
dt.size: 16139320
Término em:  2023-04-27 07:02:57.513423


In [29]:
## Normalizando as Tabelas

print('Realizando normalização da tabela. Início em: ', datetime.now())

dt['bool_cnpq'] = dt.apply(lambda d: 1 if (d['capes'] == False and d['pagtos'] > 0)
                           else 0, axis=1)
dt['bool_capes'] = dt.apply(lambda d: 1 if (d['capes'] == True and d['pagtos'] == 0)
                            else 0, axis=1)
dt['bool_ambos'] = dt.apply(lambda d: 1 if (d['capes'] == True and d['pagtos'] > 0)
                            else 0, axis=1)
dt['bool_nenhum'] = dt.apply(lambda d: 1 if (d['capes'] == False and d['pagtos'] == 0)
                                     else 0, axis=1)
dt['const'] = 1
dt = dt.loc[:,~dt.columns.duplicated()]

#Preenchendo Nan com zeros nos indicadores
dt.qty_2012 = dt.qty_2012.fillna(0)
dt.qty_2013 = dt.qty_2013.fillna(0)
dt.qty_2014 = dt.qty_2014.fillna(0)
dt.qty_2015 = dt.qty_2015.fillna(0)
dt.qty_2016 = dt.qty_2016.fillna(0)
dt.qty_2017 = dt.qty_2017.fillna(0)
dt.qty_2018 = dt.qty_2018.fillna(0)
dt.qty_2019 = dt.qty_2019.fillna(0)
dt.capes = dt.capes.fillna(0)
dt.pagtos = dt.pagtos.fillna(0)
dt.bool_cnpq = dt.bool_cnpq.fillna(0)
dt.bool_capes = dt.bool_capes.fillna(0)

num_ids = len(dt.id.unique())

print('Fim em: ', datetime.now())


Realizando normalização da tabela. Início em:  2023-04-27 13:54:15.022043
Fim em:  2023-04-27 13:54:44.181095


In [38]:
# Removendo os que tiveram duplo financiamento
dt = dt.loc[dt.bool_ambos!=True]

In [39]:
def regressão(X,y):
    try:
        # OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
        reg = sm.OLS(y, X).fit()
        # mostrando as estatísticas do modelo
        #reg.summary()
        
        
        
        #guardando a regressão na memória

        result = {
                'Indicador': indicador,
                'Erro': False,
                'Prob (F-statistic)':reg.f_pvalue, 
                'Parâmetros': reg.params, 
                "P>|t|":reg.pvalues, 
                'Standard Error': reg.bse, 
                "Covariância entre os pasrâmetros": reg.normalized_cov_params, 
                'Number of observations n.': reg.nobs, 
                "R2 ajustado": reg.rsquared_adj, 
                "Modelo":reg.model,
            }
    except:
        result = {
                'Indicador': indicador,
                'Erro': True,
                'Prob (F-statistic)':None, 
                'Parâmetros': None, 
                "P>|t|":None, 
                'Standard Error': None, 
                "Covariância entre os pasrâmetros": None, 
                'Number of observations n.': None, 
                "R2 ajustado": None, 
                "Modelo": None,
            }
    return result

In [40]:
def regressão_com_confiança (X,y,confiança = 0.05): 
    result = regressão(X,y)
    #print(result)
    while result['Erro'] == False and result['P>|t|'].max() > confiança:
        X = X.drop(columns=result['P>|t|'].idxmax())
        result = regressão(X,y)
        #print(result)
    if result['Erro'] == False and result['Prob (F-statistic)'] < confiança and result['P>|t|'].max() < confiança:
        result['Confiança'] = confiança
        return result
    else:
        result['Confiança'] = 'Erro'
    return result


In [41]:
## inicializando variáveis
lista_regressões = []
tempo_início = datetime.now()
feitos = 0
print(f'Iniciado em {tempo_início}.')
Impacto = []

#pegando lista de todos os indicadores
todos_indicadores = dt.nome.unique()

#iterando sobre cada indicador
número_total = len(todos_indicadores)
for indicador in todos_indicadores:
    
    #Realizando previsão de término
    if not feitos == 0:
        porcentagem_já_feita = (feitos/número_total)
        tempo_passado = datetime.now() - tempo_início
        tempo_por_id = tempo_passado / feitos
        tempo_restante = (número_total - feitos) * tempo_por_id
        tempo_em_que_vai_acabar = datetime.now() + tempo_restante
        print(f'{feitos}/{número_total}. {porcentagem_já_feita * 100}% feitos. Fazendo indicador: {indicador}. Acabará em {tempo_em_que_vai_acabar}. O Último demorou: {datetime.now() - tempo_último}.')
    else:
        print(f'{feitos}/{número_total}. Fazendo id: {indicador}.')
    feitos += 1
    tempo_último = datetime.now()

    #Realizando a regressão
    dados = dt.loc[dt.nome==indicador]
    X = dados[['const','qty_2013', 'bool_cnpq','bool_capes']]
    y = dados[['qty_2019']]
    lista_confianças = (0.01, 0.05)
    for confiança in lista_confianças:
        result = regressão_com_confiança(X,y, confiança)
        if not result['Confiança'] == 'Erro':
            break

    
    if not result['Erro'] and not result['Confiança'] == 'Erro':
        if not 'bool_cnpq' in result['Parâmetros'] :
            result['Parâmetros']['bool_cnpq'] = 0
        if not 'bool_capes' in result['Parâmetros'] :
            result['Parâmetros']['bool_capes'] = 0
        if not 'const' in result['Parâmetros'] :
            result['Parâmetros']['const'] = 0   
        if not 'qty_2013' in result['Parâmetros'] :
            result['Parâmetros']['qty_2013'] = 0         

        temp = {
            'Indicador': indicador,
            'Tamanho Amostra:': len(dados),
            'R2 ajustado': result['R2 ajustado'],
            'Confiança': result['Confiança'],
            'alfa': result['Parâmetros']['const'],
            'qty_2013': result['Parâmetros']['qty_2013'],
            'amostra_qty_2013': len(dados.loc[dados.bool_capes == 0].loc[dados.bool_cnpq == 0]),
            'bool_capes': result['Parâmetros']['bool_capes'],
            'amostra_capes': len(dados.loc[dados.capes == True]),
            'Impacto CAPES': result['Parâmetros']['bool_capes'] * len(dados.loc[dados.bool_capes == 1]),
            'bool_cnpq': result['Parâmetros']['bool_cnpq'],
            'amostra_cnpq': len(dados.loc[dados.bool_cnpq == 1]),
            'Impacto CNPq': result['Parâmetros']['bool_cnpq'] * len(dados.loc[dados.bool_cnpq == 1]),
            }
        Impacto.append(temp)
  
    lista_regressões.append(result)


Iniciado em 2023-04-27 13:58:24.868266.
0/130. Fazendo id: Início de Extensão Universitária.
1/130. 0.7692307692307693% feitos. Fazendo indicador: Início de Graduação. Acabará em 2023-04-27 13:59:04.699876. O Último demorou: 0:00:00.128115.
2/130. 1.5384615384615385% feitos. Fazendo indicador: Início de Mestrado. Acabará em 2023-04-27 13:58:52.973096. O Último demorou: 0:00:00.124985.
3/130. 2.307692307692308% feitos. Fazendo indicador: Obtenção do Título de Doutorado. Acabará em 2023-04-27 13:58:49.764175. O Último demorou: 0:00:00.142138.
4/130. 3.076923076923077% feitos. Fazendo indicador: Obtenção do Título de Metrado. Acabará em 2023-04-27 13:58:49.149926. O Último demorou: 0:00:00.172608.
5/130. 3.8461538461538463% feitos. Fazendo indicador: Detalhamento do Trabalho. Acabará em 2023-04-27 13:58:47.378677. O Último demorou: 0:00:00.118658.
6/130. 4.615384615384616% feitos. Fazendo indicador: Patente Registrada. Acabará em 2023-04-27 13:58:46.542384. O Último demorou: 0:00:00.12654

53/130. 40.76923076923077% feitos. Fazendo indicador: Outras Participações em Congressos. Acabará em 2023-04-27 13:58:41.353443. O Último demorou: 0:00:00.167658.
54/130. 41.53846153846154% feitos. Fazendo indicador: Participação em Banca de Doutorado. Acabará em 2023-04-27 13:58:41.403208. O Último demorou: 0:00:00.147466.
55/130. 42.30769230769231% feitos. Fazendo indicador: Participação em Banca de Exame de Qualificação. Acabará em 2023-04-27 13:58:41.364898. O Último demorou: 0:00:00.111007.
56/130. 43.07692307692308% feitos. Fazendo indicador: Participação em Banca de Mestrado. Acabará em 2023-04-27 13:58:41.337679. O Último demorou: 0:00:00.115144.
57/130. 43.84615384615385% feitos. Fazendo indicador: Participação em Congresso. Acabará em 2023-04-27 13:58:41.292854. O Último demorou: 0:00:00.106050.
58/130. 44.61538461538462% feitos. Fazendo indicador: Programa de Rádio ou TV. Acabará em 2023-04-27 13:58:41.293531. O Último demorou: 0:00:00.126660.
59/130. 45.38461538461539% feit

106/130. 81.53846153846153% feitos. Fazendo indicador: Música. Acabará em 2023-04-27 13:58:40.120810. O Último demorou: 0:00:00.102248.
107/130. 82.3076923076923% feitos. Fazendo indicador: Produção de Carta, Mapa ou Similar. Acabará em 2023-04-27 13:58:40.082768. O Último demorou: 0:00:00.085001.
108/130. 83.07692307692308% feitos. Fazendo indicador: Participação em Banca Julgadora para Livre Docência. Acabará em 2023-04-27 13:58:40.056211. O Último demorou: 0:00:00.094988.
109/130. 83.84615384615385% feitos. Fazendo indicador: Participação em Banca Julgadora para Professor Titular. Acabará em 2023-04-27 13:58:40.042293. O Último demorou: 0:00:00.105159.
110/130. 84.61538461538461% feitos. Fazendo indicador: Produção de Artes Visuais. Acabará em 2023-04-27 13:58:40.000090. O Último demorou: 0:00:00.081000.
111/130. 85.38461538461539% feitos. Fazendo indicador: Treinamento Ministrado. Acabará em 2023-04-27 13:58:39.975041. O Último demorou: 0:00:00.095017.
112/130. 86.15384615384616% f

C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1804: RuntimeWarning: divide by zero encountered in double_scalars
  return self.mse_model/self.mse_resid


In [50]:
dt.loc[dt.capes == True].loc[dt.pagtos > 0]

,index,id,pagtos,capes,indicador_tipo,indicador_grupo,indicador,qty_2012,qty_2013,qty_2014,...,qty_2019,tipo,nome,grupo,path,bool_cnpq,bool_capes,bool_ambos,bool_nenhum,const


In [12]:
with open('Regressões dos Doutorandos 2014 com CAPES e CNPq.pickle', 'wb') as f:
   pickle.dump(lista_regressões, f)

In [51]:
regressões = pd.DataFrame(lista_regressões)

In [52]:
regressões = pd.concat([regressões, regressões['Parâmetros'].apply(pd.Series), regressões['P>|t|'].apply(pd.Series), regressões['Standard Error'].apply(pd.Series)], axis=1)

In [53]:
regressões.to_excel('Regressões dos Doutorandos 2014 com CAPES e CNPq.xlsx')

In [54]:
valor_total_cnpq =  dt[['id', 'pagtos']].loc[dt['pagtos']>0].drop_duplicates().sum().pagtos

In [55]:
impacto = pd.DataFrame(Impacto)
impacto.to_excel('Impacto dos Doutorandos 2014 com CAPES e CNPq.xlsx')

In [56]:
Impacto

[{'Indicador': 'Início de Extensão Universitária',
  'Tamanho Amostra:': 12700,
  'R2 ajustado': 0.8192681908312248,
  'Confiança': 0.01,
  'alfa': 0.4514297819523318,
  'qty_2013': 1.0920956012758027,
  'amostra_qty_2013': 6515,
  'bool_capes': 0.0,
  'amostra_capes': 4938,
  'Impacto CAPES': 0.0,
  'bool_cnpq': 0.0,
  'amostra_cnpq': 1247,
  'Impacto CNPq': 0.0},
 {'Indicador': 'Início de Graduação',
  'Tamanho Amostra:': 20620,
  'R2 ajustado': 0.7810887922676701,
  'Confiança': 0.01,
  'alfa': 0.0142808150810993,
  'qty_2013': 1.0324546691754093,
  'amostra_qty_2013': 9974,
  'bool_capes': 0.014568644740729393,
  'amostra_capes': 8239,
  'Impacto CAPES': 120.03106401886947,
  'bool_cnpq': 0.0,
  'amostra_cnpq': 2407,
  'Impacto CNPq': 0.0},
 {'Indicador': 'Início de Mestrado',
  'Tamanho Amostra:': 19679,
  'R2 ajustado': 0.7229184313746507,
  'Confiança': 0.01,
  'alfa': 0.12059809093124904,
  'qty_2013': 0.8890986737761505,
  'amostra_qty_2013': 9358,
  'bool_capes': -0.004386372